In [1]:
!pip install efficientnet

In [2]:
import os
os.chdir("../")

In [5]:
import cv2
import numpy as np
import pandas as pd
from keras.models import load_model
from tqdm import tqdm
from efficientnet.keras import EfficientNetB5
df = pd.read_csv("../data/files/train.csv")

loaded_model = load_model('../saved_models/diabetic_eye_model_GPU_trained_5_epoch.h5')
test_image_names = os.listdir("../data/images/train_images")

def resmi_isle(resim_adı):
    image = cv2.imread('../data/images/train_images/'+ resim_adı)
    image = cv2.resize(image, (400,400))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    kopya = image.copy()
    kopya = cv2.cvtColor (kopya, cv2.COLOR_RGB2GRAY)
    blur = cv2.GaussianBlur (kopya, (5,5),0)
    thresh = cv2.threshold (blur, 10,255, cv2.THRESH_BINARY) [1]
    # 10 ve altında olanlar siyah üstünde olamlar beyaz
    kontur = cv2.findContours (thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # sınırları bul fotoğrafın
    kontur = kontur [0][0]
    kontur = kontur [:,0,:]
    x1 = tuple(kontur [kontur[:,0].argmin()])[0]
    y1 = tuple(kontur [kontur[:,1].argmin()]) [1]
    x2 = tuple(kontur [kontur[:,0].argmax()]) [0]
    y2 = tuple(kontur [kontur[:,1].argmax()]) [1]

    x = int (x2-x1) *4//50
    y = int(y2-y1) *5//50
    kopya2 = image.copy()
    if x2-x1 >100 and y2-y1> 100:
        kopya2 = kopya2 [y1+y: y2-y, x1+x: x2-x]
        kopya2 = cv2.resize(kopya2, (400,400))

    lab = cv2.cvtColor (kopya2, cv2.COLOR_RGB2LAB)
    l, a,b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=5.0,tileGridSize=((8,8))) # parlaklığın azaltılması
    cl = clahe.apply(l)

    limg = cv2.merge((cl,a,b))
    son = cv2.cvtColor (limg, cv2.COLOR_LAB2RGB)
    med_son = cv2.medianBlur (son, 3)
    arka_plan = cv2.medianBlur (son, 37)
    maske = cv2.addWeighted (med_son,1,arka_plan,-1,255)
    son_img = cv2.bitwise_and(maske, med_son)

    return son_img

df_dict = dict()
for i in range(len(df)):
  df_dict[df["id_code"][i]] = df["diagnosis"][i]

true_counter = 0
false_counter = 0

for img_name in tqdm(test_image_names):

  test_resim = resmi_isle(img_name)
  resim_array = test_resim.reshape((1, 400, 400, 3))
  prediction = loaded_model.predict(resim_array, verbose = 0)
  hasta_tani = np.argmax(prediction)

  img_name_strip = img_name.strip(".png")
  if df_dict[img_name_strip] == hasta_tani:
    true_counter+=1
  else:
    false_counter+=1

print(f"{true_counter}/{true_counter + false_counter} Doğru Bilindi")
print(f"{false_counter}/{true_counter + false_counter} Yanlış Bilindi")

100%|██████████| 3662/3662 [1:26:44<00:00,  1.42s/it]

1806/3662 Doğru Bilindi
1856/3662 Yanlış Bilindi


cpu trained 1 epoch = 5/10 Yanlış Bilindi
GPU trained 1 epoch = 5/10 Yanlış Bilindi
